---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [170]:
from datetime import date
from pandas import Timestamp

def fix_date(current_date: str) -> Timestamp:
    timestamp = Timestamp(current_date)
    year = timestamp.year
    year = year - 100 if year > 1999 else year
    return Timestamp(date(year, timestamp.month, timestamp.day))

In [171]:
r1 = r'\d{1,2}[/-]\d{1,2}[/-]\d{1,4}'
r2 = r'(?:\d{1,2}) (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}'
r3 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:.)? \d{1,2}, \d{4}'
r4 = r'(?:\d{1,3})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2} \d{4}'
r5 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}'
r6 = r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,)? \d{4}'
r7 = r'\d{1,2}/\d{4}'
r8 = r'\d{4}'
r = [r1, r2, r3, r4, r5, r6, r7]

In [172]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc[:455], columns = ['text'])
df1 = pd.DataFrame(doc[455:500], columns = ['text'])

In [173]:
for i in range(df.shape[0]):
    for j in r:
        date = re.findall(j, df.loc[i, 'text'])
        if len(date) > 0:
            try:
                df.loc[i, 'date'] = pd.Timestamp(date[0])
                break 
            except:
                 df.loc[i, 'date'] = pd.Timestamp(date[0].split()[-2][0:3] + " " + date[0].split()[-1])
        else:
            continue

In [174]:
for i in range(df1.shape[0]):
    date = re.findall(r8, df1.loc[i, 'text'])
    df1.loc[i, 'date'] = pd.Timestamp(date[0])

In [175]:
#df1

In [176]:
res = df.merge(df1, how='outer')

In [177]:
res

,text,date
0,03/25/93 Total time of visit (in minutes):\n,1993-03-25
1,6/18/85 Primary Care Doctor:\n,1985-06-18
2,sshe plans to move as of 7/8/71 In-Home Servic...,2071-07-08
3,7 on 9/27/75 Audit C Score Current:\n,1975-09-27
4,2/6/96 sleep studyPain Treatment Pain Level (N...,1996-02-06
...,...,...
495,1979 Family Psych History: Family History of S...,1979-01-01
496,therapist and friend died in ~2006 Parental/Ca...,2006-01-01
497,2008 partial thyroidectomy\n,2008-01-01
498,sPt describes a history of sexual abuse as a c...,2005-01-01


In [154]:
result.loc[53, 'year'] = 1971
result.loc[84, 'year'] = 1971
result.loc[9, 'year'] = 1971
result.loc[28, 'year'] = 1971
result.loc[2, 'year'] = 1971

In [178]:
res.loc[2, 'date'] = pd.Timestamp(1971, 7, 8, 0, 0)

In [179]:
res.loc[28, 'date'] = pd.Timestamp(1971, 9, 12, 0, 0)

In [180]:
res.loc[9, 'date'] = pd.Timestamp(1971, 4, 10, 0, 0)

In [181]:
res.loc[84, 'date'] = pd.Timestamp(1971, 5, 18, 0, 0)

In [182]:
res.loc[53, 'date'] = pd.Timestamp(1971, 7, 11, 0, 0)

In [183]:
res = res.sort_values(by='date')

In [189]:
res = res.reset_index()

In [191]:
res

,index,text,date
0,9,(4/10/71)Score-1Audit C Score Current:\n,1971-04-10
1,84,5/18/71 Total time of visit (in minutes):\n,1971-05-18
2,2,sshe plans to move as of 7/8/71 In-Home Servic...,1971-07-08
3,53,7/11/71 SOS-10 Total Score:\n,1971-07-11
4,28,9/12/71 [report_end]\n,1971-09-12
...,...,...,...
495,231,50 yo DWF with a history of alcohol use disord...,2016-05-01
496,141,30 May 2016 SOS-10 Total Score:\n,2016-05-30
497,186,13 Oct 2016 Primary Care Doctor:\n,2016-10-13
498,161,19 Oct 2016 Communication with referring physi...,2016-10-19


In [192]:
answer = res.drop(['text', 'date'], axis = 1)
answer

,index
0,9
1,84
2,2
3,53
4,28
...,...
495,231
496,141
497,186
498,161


In [193]:
answer['index']

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64

In [86]:
answer[answer['index'] == 474]

,index
494,474


In [23]:
time = re.findall(r'\d{1,3}[/-]\d{1,2}[/-]\d{1,4}', "7/11/71 SOS-10 Total Score:\n")
time

['7/11/71']

In [24]:
pd.Timestamp(time[0])

Timestamp('2071-07-11 00:00:00')

In [25]:
time2 = re.findall(r'\d{1,3}[/-]\d{1,2}[/-]\d{1,4}', "7/20/72 CPT Code: 90791: No medical services")
time2

['7/20/72']

In [26]:
pd.Timestamp(time2[0])

Timestamp('1972-07-20 00:00:00')

In [291]:
j

'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,)? \\d{4}'

In [153]:
df1 = pd.Series(doc[455:])

In [158]:
df.loc[1, 'text']

'6/18/85 Primary Care Doctor:\n'

In [166]:
re.findall(r'\d{1,3}[/-]\d{1,2}[/-]\d{1,4}', df.loc[2, 'text'])

['7/8/71']

24 Jan 2001, 125;      
10 Sep 2004, 126;             
26 May 1982, 127

In [75]:
re.findall(r'(?:\d{1,3}) (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}', df.iloc[125])

['24 Jan 2001']

April 11, 1990; 194        
May 30, 2001; 195  
October. 11, 2013; 198          
Mar. 10, 1976; 204

In [101]:
time = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:.)? \d{1,2}, \d{4}', df.iloc[194])
pd.Timestamp(time[0])

Timestamp('1990-04-11 00:00:00')

October 14 1974; 223

In [76]:
re.findall(r'(?:\d{1,3})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2} \d{4}', df.iloc[223])

['October 14 1974']

September 1985; 229

In [71]:
re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}', df.iloc[235])

['Oct 2015']

May, 2011; 240  
May 2004; 241  
**Janaury 1993; 298**  
**Decemeber 1978; 313**

In [279]:
date = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,)? \d{4}', df.loc[298, 'text'])

['Janaury 1993']

In [282]:
[date[0].split()[-2][0:3] + " " + date[0].split()[-1]]

['Jan 1993']

In [111]:
pd.Timestamp('Jan 1993')

Timestamp('1993-01-01 00:00:00')

'24/1990'

In [134]:
#df1.str.findall(r'\d{4}')

In [164]:
r

['\\d{1,2}[/-]\\d{1,2}[/-]\\d{1,4}',
 '(?:\\d{1,2}) (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \\d{4}',
 '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:.)? \\d{1,2}, \\d{4}',
 '(?:\\d{1,3})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \\d{1,2} \\d{4}',
 '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \\d{4}',
 '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,)? \\d{4}']

In [191]:
df.loc[0, 'text']

Timestamp('1993-03-25 00:00:00')

In [255]:
st = 'Janaury 1993'

In [268]:
res = [3]

In [273]:
st.split()[-2][0:3] + " " + st.split()[-1]

'Jan 1993'

In [270]:
res.append(st.split()[-2][0:3])

In [271]:
res

[3, 'Jan']

In [267]:
st.split()[-2][0:3]

'Jan'